###  Estimating the effect of new editors/owners on Newspaper Coverage

In [ ]:
# structural drift panel creation

import pandas as pd
import json
import numpy as np

# Load data sources
newspapers = pd.read_csv('data/final_list.csv')
with open('data/topic_counts.json', 'r') as f:
    topic_data = json.load(f)

TOPICS = [
    'labor_workers', 'politics_elections', 'congress_government',
    'business_commerce', 'railroads_transportation', 'agriculture_farming',
    'courts_law', 'finance_money', 'immigration_foreign', 'crime_police'
]

# =============================================================================
# Step 1: Build raw panel with topic rates per 1,000 headlines
# =============================================================================
records = []
for year, papers in topic_data.items():
    for paper_name, data in papers.items():
        if 'topic_counts' in data and 'total_headlines' in data:
            total = data['total_headlines']
            if total > 0:
                record = {
                    'year': int(year),
                    'newspapers_all_years_name': paper_name.lower().strip()
                }
                for topic in TOPICS:
                    count = data['topic_counts'].get(topic, 0)
                    record[topic] = (count / total) * 1000
                records.append(record)

panel = pd.DataFrame(records)

# Merge with metadata
newspapers['newspapers_all_years_name'] = newspapers['newspapers_all_years_name'].str.lower().str.strip()
panel = panel.merge(
    newspapers[['newspapers_all_years_name', 'master_id', 'master_name', 'publisher_change_year']],
    on='newspapers_all_years_name', how='left'
)

# =============================================================================
# Step 2: Identify treated vs control; find median treatment year
# Correct treatment year by subtracting 1
# =============================================================================
panel['is_treated'] = panel['publisher_change_year'].notna()
panel['publisher_change_year'] = panel['publisher_change_year'] - 1
median_treat_year = panel.loc[panel['is_treated'], 'publisher_change_year'].median()
print(f"Median treatment year among treated papers: {median_treat_year}")

# =============================================================================
# Step 3: Define anchor cutoff year for each paper
# Treated: their specific treatment year
# Control: the median treatment year
# =============================================================================
panel['anchor_cutoff'] = np.where(
    panel['is_treated'],
    panel['publisher_change_year'],
    median_treat_year
)

# =============================================================================
# Step 4: Filter papers with at least 3 pre-treatment years
# (using the ADJUSTED anchor cutoff)
# =============================================================================
pre_counts = panel[panel['year'] < panel['anchor_cutoff']].groupby('master_id').size()
valid_papers = pre_counts[pre_counts >= 3].index
panel = panel[panel['master_id'].isin(valid_papers)].copy()
print(f"Papers with ≥3 pre-treatment years (adjusted): {len(valid_papers)}")

# =============================================================================
# Step 5: Calculate anchor vector (pre-treatment average) for each paper
# Using the ADJUSTED cutoff
# =============================================================================
pre_panel = panel[panel['year'] < panel['anchor_cutoff']]
anchors = pre_panel.groupby('master_id')[TOPICS].mean()
anchors.columns = [f'anchor_{t}' for t in TOPICS]

panel = panel.merge(anchors, on='master_id', how='left')

# =============================================================================
# Step 6: Calculate Y_it = Euclidean distance from anchor
# =============================================================================
def calc_drift(row):
    sq_diffs = sum((row[t] - row[f'anchor_{t}'])**2 for t in TOPICS)
    return np.sqrt(sq_diffs)

panel['Y_it'] = panel.apply(calc_drift, axis=1)

# =============================================================================
# Step 7: Create Post_it dummy (relative to ADJUSTED anchor cutoff)
# =============================================================================
panel['Post_it'] = (panel['year'] >= panel['anchor_cutoff']).astype(int)

# =============================================================================
# Step 8: Build final output table
# =============================================================================
output = panel[[
    'master_id', 'master_name', 'year', 'is_treated', 
    'anchor_cutoff', 'Post_it', 'Y_it'
]].rename(columns={
    'master_id': 'Newspaper_ID',
    'master_name': 'Newspaper_Name',
    'year': 'Year',
    'anchor_cutoff': 'Anchor_Cutoff_Year'
}).sort_values(['Newspaper_ID', 'Year']).reset_index(drop=True)

# Display diagnostics
print("\nSample of final panel:\n")
print(output.head(20).to_string(index=False))

print(f"\n--- Panel Summary ---")
print(f"Total observations: {len(output)}")
print(f"Unique newspapers: {output['Newspaper_ID'].nunique()}")
print(f"Treated papers: {output[output['is_treated']]['Newspaper_ID'].nunique()}")
print(f"Control papers: {output[~output['is_treated']]['Newspaper_ID'].nunique()}")
print(f"Pre-treatment obs: {(output['Post_it'] == 0).sum()}")
print(f"Post-treatment obs: {(output['Post_it'] == 1).sum()}")

# Save
output.to_csv('data/panel_structural_drift.csv', index=False)
print("\nSaved to 'panel_structural_drift.csv'")

In [ ]:
# regression

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

# Load the structural drift panel
panel = pd.read_csv('data/panel_structural_drift.csv')

# =============================================================================
# Model 1: Static DiD
# Y_it = α_i + δ_t + β(Post_it) + ε_it
# Tests: Did treated papers drift further from baseline than controls?
# =============================================================================
reg_data = panel[['Newspaper_ID', 'Year', 'Post_it', 'Y_it', 'is_treated']].dropna()

static_model = smf.ols(
    'Y_it ~ Post_it + C(Newspaper_ID) + C(Year)',
    data=reg_data
).fit(cov_type='cluster', cov_kwds={'groups': reg_data['Newspaper_ID']})

print("=" * 70)
print("MODEL 1: STATIC DiD — Average Treatment Effect on Structural Drift")
print("=" * 70)
print(f"\nβ (Post_it):     {static_model.params['Post_it']:.4f}")
print(f"Std Error:       {static_model.bse['Post_it']:.4f}")
print(f"95% CI:          [{static_model.conf_int().loc['Post_it', 0]:.4f}, {static_model.conf_int().loc['Post_it', 1]:.4f}]")
print(f"P-value:         {static_model.pvalues['Post_it']:.4f}")
print(f"R-squared:       {static_model.rsquared:.4f}")
print(f"Observations:    {static_model.nobs:.0f}")

# =============================================================================
# Model 2: Event Study (Dynamic Effects)
# Y_it = α_i + δ_t + Σ β_k D^k_it + ε_it
# Shows timing of drift: immediate vs gradual?
# =============================================================================
K_MIN, K_MAX = -5, 5

# Calculate time relative to anchor cutoff
panel['Time_to_Treat'] = panel['Year'] - panel['Anchor_Cutoff_Year']

def bin_time(k):
    if pd.isna(k): return None
    if k < K_MIN: return f'k_{K_MIN}'
    if k > K_MAX: return f'k_{K_MAX}'
    return f'k_{int(k)}'

panel['event_time'] = panel['Time_to_Treat'].apply(bin_time)
panel['event_time'] = pd.Categorical(
    panel['event_time'],
    categories=[f'k_{k}' for k in range(K_MIN, K_MAX + 1)]
)

reg_data_es = panel[['Newspaper_ID', 'Year', 'event_time', 'Y_it']].dropna()

event_model = smf.ols(
    'Y_it ~ C(event_time, Treatment("k_-1")) + C(Newspaper_ID) + C(Year)',
    data=reg_data_es
).fit(cov_type='cluster', cov_kwds={'groups': reg_data_es['Newspaper_ID']})

# Extract coefficients for plotting
coefs = []
for k in range(K_MIN, K_MAX + 1):
    if k == -1:
        coefs.append({'k': k, 'beta': 0, 'ci_low': 0, 'ci_high': 0, 'pval': np.nan})
    else:
        param = f'C(event_time, Treatment("k_-1"))[T.k_{k}]'
        if param in event_model.params:
            coefs.append({
                'k': k,
                'beta': event_model.params[param],
                'ci_low': event_model.conf_int().loc[param, 0],
                'ci_high': event_model.conf_int().loc[param, 1],
                'pval': event_model.pvalues[param]
            })

coef_df = pd.DataFrame(coefs)

print("\n" + "=" * 70)
print("MODEL 2: EVENT STUDY — Dynamic Path of Structural Drift")
print("=" * 70)
print(coef_df.to_string(index=False, float_format='{:.4f}'.format))

# Plot
fig, ax = plt.subplots(figsize=(10, 6))

ax.errorbar(coef_df['k'], coef_df['beta'],
            yerr=[coef_df['beta'] - coef_df['ci_low'],
                  coef_df['ci_high'] - coef_df['beta']],
            fmt='o', capsize=4, color='steelblue', markersize=8)

ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax.axvline(x=-0.5, color='red', linestyle='--', linewidth=1.5, label='Treatment')
ax.fill_betweenx(ax.get_ylim(), K_MIN - 0.5, -0.5, alpha=0.1, color='gray', label='Pre-treatment')

ax.set_xlabel('Years Relative to Anchor Cutoff (k)', fontsize=12)
ax.set_ylabel('Drift from Historical Baseline (βₖ)', fontsize=12)
ax.set_title('Event Study: Structural Drift from Pre-Treatment Identity', fontsize=14)
ax.set_xticks(range(K_MIN, K_MAX + 1))
ax.legend()

plt.tight_layout()
plt.savefig('figures/structural_drift_event_study.png', dpi=150)
plt.show()